In [3]:
import os

print("현재 작업 디렉토리:", os.getcwd())
os.chdir('/workspaces/gpt-recruit-rag/packages/gpt-recruit-rag')  # 프로젝트 루트 디렉토리로 변경
print("변경 후 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: /workspaces/gpt-recruit-rag/packages/gpt-recruit-rag
변경 후 작업 디렉토리: /workspaces/gpt-recruit-rag/packages


In [4]:
import os
from dotenv import dotenv_values

config = dotenv_values(".env")

os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]

In [28]:
from operator import itemgetter
from typing import List, Optional, Tuple

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_elasticsearch import ElasticsearchStore
from langchain_community.embeddings import HuggingFaceHubEmbeddings

from prompts import CONDENSE_QUESTION_PROMPT, DOCUMENT_PROMPT, LLM_CONTEXT_PROMPT


llm = ChatOpenAI(
    base_url='http://ruoserver.iptime.org:11434/v1/',
    model='EEVE-Ko-Q8'
)

embedding = HuggingFaceHubEmbeddings(model=config["EMBEDDING_URI"])
# vectorstore = ElasticsearchStore(
#     es_url=config["ES_URI"],
#     es_api_key=config["ES_API_KEY"],
#     index_name="wanted_job_detail_index_v1",
#     vector_query_field='embedding',
#     query_field='metadata.position',
#     embedding=embedding,
#     strategy=ElasticsearchStore.ApproxRetrievalStrategy(
#         hybrid=True,
#     )
# )   # ApproxRetrievalStrategy 사용 (유료 플랜)

vectorstore = ElasticsearchStore(
    es_url=config["ES_URI"],
    es_api_key=config["ES_API_KEY"],
    index_name="wanted_job_detail_index_v1",
    vector_query_field='embedding',
    query_field='metadata.position',
    embedding=embedding
)

retriever = vectorstore.as_retriever()


def _combine_documents(
    docs, document_prompt=DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer


class ChainInput(BaseModel):
    chat_history: Optional[List[BaseMessage]] = Field(
        description="Previous chat messages."
    )
    question: str = Field(..., description="The question to answer.")


_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
)

_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}

chain = _inputs | _context | LLM_CONTEXT_PROMPT | llm | StrOutputParser()

chain = chain.with_types(input_type=ChainInput)


In [20]:
questions = [
    "파이썬 (python) 개발자 채용 정보를 알려주세요.",
    "오케스트로 회사의 채용 정보를 알려주세요.",
    "데이터 엔지니어 채용 회사는 어디인가요?",
    "IT 관련 채용 회사는 어디인가요?",
    "디자인 관련 채용 회사는 어디인가요?",
    "마케팅 관련 채용 회사는 어디인가요?",
    "데이터 관련 채용 회사는 어디인가요?",
]

In [4]:
response = chain.invoke(
    {
        "question": questions[0],
        "chat_history": [],
    }
)
print(response)

1. 딥인사이트 (출처: NAME: 딥인사이트): [질문] IT 분야에서의 채용 회사들은 어디인가요?

---
딥인사이트는 정보 기술(IT) 분야에서 숙련된 인재를 찾는 회사에 서비스를 제공하는 전문 IT 채용 기업입니다. 다양한 분야의 최고급 소프트웨어 개발자, 디지털 마케팅 전문가, 데이터 분석가 등 뛰어난 인력을 연결하는 데 초점을 맞추고 있습니다. 이 회사는 기업들이 각자의 독특한 요구사항과 목표에 맞는 숙련된 인재를 찾아 팀을 강화할 수 있도록 돕습니다.
---

2. 코딧(CODITCorp.) (출처: NAME: 코딧(CODITCorp.)): [질문] IT 분야에서의 채용 회사들은 어디인가요?

---
코딧은 정보 기술 전문가를 찾는 회사에 서비스를 제공하는 선도적인 IT 채용 기업으로, 소프트웨어 개발자부터 사이버보안 전문가, 데이터 과학자까지 다양한 인재를 구인하는 데 도움을 줍니다. 이 회사는 글로벌하게 운영되며 전 세계의 기업들이 숙련된 인력을 효과적으로 찾을 수 있도록 도와 팀을 강화할 수 있습니다.
---

3. 스마일게이트홀딩스 슈퍼크리에이티브 지점 (출처: NAME: 스마일게이트홀딩스 슈퍼크리에이티브 지점): [질문] IT 분야에서의 채용 회사들은 어디인가요?

---
스마일게이트홀딩스는 게임 개발과 기술 분야에서 인재를 찾는 회사에 전문 서비스를 제공하는 유망한 IT 채용 기업입니다. 이 회사는 스마일게이트홀딩스의 자회사인 슈퍼크리에이티브의 지점으로, 비디오 게임을 만들고 배포하는 데 필요한 기술과 경험을 가진 인재를 구인하는데 중점을 두고 있습니다.
---

4. 노타 (출처: NAME: 노타): [질문] IT 분야에서의 채용 회사들은 어디인가요?

---
노타는 정보 기술 분야에서 숙련된 인력을 찾는 회사에 전문 서비스를 제공하는 선도적인 IT 채용 기업입니다. 소프트웨어 개발자, 데이터 분석가, 사이버보안 전문가 등 다양한 인재를 구인하는데 도움을 주며, 기업들은 이 회사의 전문성을 활용하여 자신의 독특한 요구사항에 맞는 최적의

In [26]:
follow_up_question = "파이썬 개발자를 채용하는 회사의 채용공고를 확인하고, 그 중 한 곳 회사의 적합한 한국식 자기소개서를 작성해주세요."

In [5]:
response = chain.invoke(
    {
        "question": follow_up_question,
        "chat_history": [
            questions[0],
            response,
        ],
    }
)

print(response)

1. 스마일게이트홀딩스 슈퍼크리에이티브 지점 (PASSAGE): 이 회사는 모바일 및 콘솔 게임 개발을 위한 '게임 디자이너', '게임 프로그래머' 그리고 다양한 분야의 'QA 테스터'를 채용하고 있습니다. 이들은 훌륭한 커뮤니케이션 기술, 강한 문제해결 능력, 게임 산업에 대한 열정을 가진 사람을 찾고 있습니다 (출처: [1]).
2. 베이글코드(Bagelcode) (PASSAGE): 그들은 모바일 게임에 특화된 '게임 디자이너', '게임 프로그래머' 및 'QA 테스터'를 찾고 있습니다. 또한 마케팅 분야에는 '마케터/콘텐츠 기획자'와 사업 개발 분야에서 '사업 개발 매니저(BD)'가 필요합니다 (출처: [2]).
3. 메디테라피 (PASSAGE): 이 회사는 제품 관리 분야에서 '제품 소유자'를 찾고 있습니다. 또한 게임 디자인과 프로그래밍을 위한 전문가들을 필요로 하며, 모바일 게임에 특화된 '게임 디자이너', '게임 프로그래머' 그리고 QA 분야에서의 'QA 테스터'를 원합니다 (출처: [3]).
4. 코딧(CODITCorp.) (PASSAGE): 게임 개발 분야에서 '게임 디자이너', '게임 아티스트', '게임 프로그래머' 그리고 모바일 게임의 질 관리를 위한 'QA 테스터'의 지원을 받고 있습니다. 또한, 그들은 마케팅 팀에 대한 '콘텐츠 기획자', 사업 개발 분야의 '사업 개발 매니저(BD)', 그리고 그들의 인사 부서에서 'HR 채용 담당자'도 찾고 있습니다 (출처: [4]).


In [29]:
chunks = []
async for chunk in chain.astream(
    {
        "question": questions[0],
        "chat_history": [],
    }
):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

AuthorizationException: AuthorizationException(403, 'security_exception', 'current license is non-compliant for [Reciprocal Rank Fusion (RRF)]')

In [23]:
response = ''.join(chunks)

In [27]:
chunks2 = []
async for chunk in chain.astream(
    {
        "question": follow_up_question,
        "chat_history": [
            questions[0],
            response,
        ],
    }
):
    chunks2.append(chunk)
    print(chunk, end="", flush=True)

회사명: 에잇퍼센트 (8Percent)
직무명: 파이썬 개발자
제목: 파이썬 개발자로서의 열정 및 전문성과 함께하는 에잇퍼센트 입사를 희망하며

안녕하세요. 존경하는 여러분,

저는 유연한 문제 해결 능력을 가진 파이썬 전문 개발자로, 에잇퍼센트에 합류하고자 열정을 가득 담아 지원합니다. 오픈소스 기술 채택을 통해 혁신을 주도하고 긍정적인 영향을 끼치는 데 전념하는 에잇퍼센트의 가치와 제 개인적 신념이 완벽하게 부합한다고 믿으며 이 자리가 저의 기술과 경험과 잘 어울린다고 확신합니다.

다양한 산업 분야에서 근무하며 소프트웨어 솔루션을 개발하는 방대한 경력을 바탕으로, 파이썬 개발에 대한 열정을 키우고 최신 프레임워크와 도구에 능숙해졌습니다. 복잡한 시스템을 이해하고 고성능의 확장 가능한 애플리케이션을 만드는 데에 있어 탁월한 실력을 보유하고 있습니다. 또한 애자일 방법론을 준수하며 팀워크와 협업 정신을 소중히 여깁니다.

프로젝트 관리, API 통합 및 소프트웨어 아키텍처에 대한 제 경험은 에잇퍼센트의 현 기술 스택에 큰 가치가 될 것이며, 새로운 개발자들이 성장하도록 돕고 조직 전체의 생산성과 효율성에 기여하고자 합니다.

에잇퍼센트에서는 윤리적이고 사회적 책임을 다하는 기업으로서 소프트웨어 솔루션을 제공함으로써 사회에 실질적인 차이를 만들어 낼 수 있는 기회가 있다고 생각합니다. 에잇퍼센트의 팀에 합류하여 제 기술과 전문성을 기여하고 혁신과 번영의 문화를 계속해서 증진시키고자 합니다.

제 자격, 열정 및 파이썬 개발 분야의 전문 지식을 고려하여 이 포지션에 지원하게 되었습니다. 이 기회를 주셔서 감사하며, 저를 더 잘 알아가고 에잇퍼센트에 기여할 수 있는 방안을 논의하기를 기대합니다.

감사합니다,

[당신의 이름]